In [1]:
import pandas as pd
import altair as alt
import vega_datasets
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
hatecrimes = pd.read_csv("../data/hate_crime.csv")
hatecrimes

,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,...,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
1,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
2,45,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,2,Aggravated Assault;Murder and Nonnegligent Man...,2.0,Residence/Home,Anti-White,Individual,M,S
3,46,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,Not Specified,1,Intimidation,1.0,Residence/Home,Anti-White,Individual,S,S
4,47,1991,AR0670000,Sevier,NaN,County,AR,Arkansas,West South Central,South,...,White,Not Specified,1,Intimidation,1.0,School/College,Anti-Black or African American,Individual,S,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253771,1513747,2023,WY0190100,Green River,NaN,City,WY,Wyoming,Mountain,West,...,White,Not Hispanic or Latino,1,Intimidation,1.0,School-Elementary/Secondary,Anti-Jewish,Individual,S,S
253772,1517229,2023,WY0130100,Casper,NaN,City,WY,Wyoming,Mountain,West,...,White,Not Hispanic or Latino,1,Aggravated Assault,1.0,Parking/Drop Lot/Garage,Anti-Arab;Anti-Islamic (Muslim),Individual,S,M
253773,1517231,2023,WY0110100,Cheyenne,NaN,City,WY,Wyoming,Mountain,West,...,White,Not Hispanic or Latino,1,Shoplifting,0.0,Grocery/Supermarket,Anti-Jehovah's Witness,Business,S,S
253774,1519468,2023,WY0150000,Park,NaN,County,WY,Wyoming,Mountain,West,...,Not Specified,Not Specified,1,All Other Larceny,1.0,Residence/Home,Anti-White,Individual,S,S


In [ ]:
alt.Chart(hatecrimes).mark_line().encode(
        x='data_year:O',
        y = 'count():N'
    ).display()

In [ ]:
alt.Chart(hatecrimes).mark_line().encode(
        x='data_year:O',
        y = 'count():N',
        color='bias_desc:N'
    ).display()

In [ ]:
# Cleaning pandas 'bias description' variable
hatecrimes.groupby(['bias_desc']).size().sort_values(ascending=False).head(10)
# Most popular hate crimes are anti Black, Jewish, White, Gay, Hispanic, Other Race, LGBT, Asian, Multi-Race, Lesbian

In [ ]:
top10 = ["Anti-Black or African American", "Anti-Jewish", "Anti-White", "Anti-Gay (Male)", "Anti-Hispanic or Latino", "Anti-Other Race/Ethnicity/Ancestry","Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)", "Anti-Asian", "Anti-Multiple Races, Group", "Anti-Lesbian (Female)"]
hatecrimestop10 = hatecrimes.loc[hatecrimes['bias_desc'].isin(top10)]
alt.Chart(hatecrimestop10).mark_line().encode(
        x='data_year:O',
        y = 'count():N',
        color='bias_desc:N'
    ).display()

In [ ]:
# Here we can see that Anti-Black hate crimes are far more prevalent than other hate crimes, but have decreased from 2022-2023. Anti-Jewish hate crimes on the other hand, have risen since 2021

In [4]:
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature='states')

In [12]:
alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)

alt.Chart(...)

In [5]:
pop = data.population_engineers_hurricanes(usecols=['state', 'id'])

In [6]:
hatecrimes_w_ids = hatecrimes.merge(pop, left_on='state_name', right_on='state')

In [17]:
hatecrimes_w_ids
state_counts = hatecrimes_w_ids.groupby(['id'], as_index=False).size()
state_counts

,id,size
0,1,900
1,2,283
2,4,7452
3,5,1174
4,6,42066
5,8,5202
6,9,3929
7,10,971
8,11,2109
9,12,5789


In [18]:
states = alt.topo_feature(data.us_10m.url, 'states')
alt.Chart(states).mark_geoshape().encode(
    color='size:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_counts, 'id', list(state_counts.columns))
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

In [ ]:
# Total heatmap of all hatecrimes in continental US -- consider adjusting for population